# III) Création d'une ontologie

Pour ce faire, nous utiliserons le package `owlready2` qui permet de créer des ontologies en python. (https://linuxfr.org/news/owlready-un-module-python-pour-manipuler-les-ontologies-owl)
Les données seront extraite de la base de données de WikiData, nous interrogerons cette base de données par l'API mis à disposition par la librairie `wikibaseintegrator` (https://github.com/LeMyst/WikibaseIntegrator#execute-sparql-queries).

Commençons par installer les packages nécessaires

In [1]:
# !pip install owlready2
# !pip install wikibaseintegrator

Récupérons les classes et sous-classes de la classe Animalia du fichier `animals_id_name_subClass_sub2Class.csv` que nous avons créé précédemment. Nous avons décité de se limiter à deux sous-classes pour des raisons de temps de récupération des informations de l'API.
De plus, nous rencontrons un problème quand les sous classes ne sont pas définies dans WikiData, nous avons donc décidé de ne pas les prendre en compte, et de créer une sous-classe d'animalia appelé sans famille.

In [2]:
import pandas as pd

df = pd.read_csv("animals_id_name_subClass_sub2Class.csv")

tableau_animaux = df['animalLabel'].values.tolist()

# Sélectionner les colonnes spécifiques
selected_columns = ["subclass2Label", "subclassLabel"]
selected_data = df[selected_columns]

# Transformer les colonnes en un tableau de la forme [[subclass2Label], [subclassLabel]]
tableau = selected_data.values.T.tolist()

# Afficher le résultat
print(tableau_animaux)

['0', '0', '0', '0', '0', '0', '0', '0', 'lion', '0', '0', '0', '0', '0', '0', 'African elephant', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'cheetah', '0', '0', '0', '0', 'leopard', 'cougar', 'jaguar', 'brown bear', '0', '0', 'electric ray', '0', '0', '0', '0', '0', 'broccoli', '0', '0', '0', 'Indian elephant', '0', '0', '0', '0', '0', '0']


On récupère la liste des images de l'évaluation, si elle est reconnue ou non, si oui 1, sinon 0.

In [3]:
image_reconnue = pd.read_csv("./imagesReconnue.csv")

image_reconnue.head()

# Le transformer en tableau de la forme [[image, reconnue], ...]
tableau_image_reconnue = image_reconnue.values.tolist()

# Afficher le résultat
print(tableau_image_reconnue)

[['n10_01', 'lion', 'big_cat', 1], ['n20_02', 'lion', 'big_cat', 0]]


Concernant la contrainte de la propriété possède pour les sous-classes d'Animalia, c'est un concept avancé dans OWL appelé "Restrictions sur les propriétés". Pour répondre au mieux à cette contrainte, nous allons utiliser AllDisjoint et AllDifferent.

In [4]:
import numpy as np
from owlready2 import Thing, AllDisjoint
import owlready2 as owl

onto = owl.get_ontology("http://example65118.com/animalia.owl")

# Définir la classe racine Animalia
with onto:
    class Animalia(Thing):
        pass


# Définir les sous-classes
with onto:
    for i in range(len(tableau)):
        # Supprimer les zéros
        tableau_sans_zeros = [x for x in tableau[i] if x != '0']
        # Supprimer les doublons
        tableau_sans_doublons = np.unique(tableau_sans_zeros)
        for j in range(len(tableau_sans_doublons)):
            if i == 0: # cela dépend de Animalia
                nom_classe = tableau_sans_doublons[j].replace(" ", "_") # Remplacer les espaces par des underscores
                # Créer une classe dynamiquement
                classe = type(nom_classe, (Animalia,), {})
            else: # c'est une sous-classe du précédent
                indices_ssClass = [i for i, x in enumerate(tableau[i]) if x == tableau_sans_doublons[j]] # Récupérer les indices des sous-sous-classes
                if len(indices_ssClass) == 1: # Alors on crée sans réfléchir la sous classe
                    nom_sous_classe = tableau_sans_doublons[j].replace(" ", "_")  # Remplacer les espaces par des underscores
                    nom_classe = tableau[i-1][indices_ssClass[0]].replace(" ", "_")  # Remplacer les espaces par des underscores
                    # Créer une classe dynamiquement
                    classe = type(nom_sous_classe, (getattr(onto, nom_classe),), {})
                else: # Alors on crée une classe en vérifiant s'il y a des doublons
                    suppresion_doublons = []
                    for k in range(len(indices_ssClass)):
                        suppresion_doublons.append([indices_ssClass[k], tableau[i][indices_ssClass[k]]])
                    uniques = [sublist for i, sublist in enumerate(suppresion_doublons) if sublist[1] not in [x[1] for x in suppresion_doublons[:i]]]
                    for l in range(len(uniques)):
                        nom_sous_classe = uniques[l][1].replace(" ", "_")  # Remplacer les espaces par des underscores
                        nom_classe = tableau[i-1][uniques[l][0]].replace(" ", "_")  # Remplacer les espaces par des underscores
                        # Créer une classe dynamiquement
                        classe = type(nom_sous_classe, (getattr(onto, nom_classe),), {})

# Définir les classes orphelines
with onto:
    class SansFamille(Animalia):
        pass

# Définition de la classe image avec son nom et si elle est reconnue ou non
with onto:
    class Image(Thing):
        pass

    class Reconnue(Image >> bool):
        pass

# Définir les exclusions, toutes les classes d'un même niveau sont disjointes
with onto:
    for i in range(len(tableau)):
        # Supprimer les zéros
        tableau_sans_zeros = [x for x in tableau[i] if x != '0']
        # Supprimer les doublons
        tableau_sans_doublons = np.unique(tableau_sans_zeros)
        # Créer une liste de classes
        liste_classes = []
        for j in range(len(tableau_sans_doublons)):
            nom_classe = tableau_sans_doublons[j].replace(" ", "_")  # Remplacer les espaces par des underscores
            liste_classes.append(getattr(onto, nom_classe))
        # Créer une liste de classes disjointes
        AllDisjoint(liste_classes)

# Définir les différentes propriétés
with onto:
    class CaracteristiqueMorphologique(Thing):
        pass

    class Patte(CaracteristiqueMorphologique):
        pass

    class Museau(CaracteristiqueMorphologique):
        pass

    class Aile(CaracteristiqueMorphologique):
        pass

    class Bec(CaracteristiqueMorphologique):
        pass

    class Queue(CaracteristiqueMorphologique):
        pass

    class Poil(CaracteristiqueMorphologique):
        pass

    class Plume(CaracteristiqueMorphologique):
        pass

# Pour définir les associations, on utilise la propriété contient, qui permet de relier une image à un animal, et on lui précisera si l'image a bien été reconnue, avec la propriété possède, qui permet de relier un animal à une caractéristique morphologique.
with onto:
    class Contient(owl.ObjectProperty):
        domain = [Image]
        range = [Animalia]
        pass

    class Possede(owl.ObjectProperty):
        domain    = [Animalia]
        range     = [CaracteristiqueMorphologique]
        pass


for i in range(len(tableau[-1])):
    if tableau[-1][i] == '0':
        nom_animal = tableau[0][i].replace(" ", "_")  # Remplacer les espaces par des underscores
        animal = SansFamille(nom_animal)
    else:
        nom_classe = tableau[-1][i].replace(" ", "_")  # Remplacer les espaces par des underscores
        nom_animal = tableau_animaux[i].replace(" ", "_")  # Remplacer les espaces par des underscores
        # Accédez à la classe spécifique
        classe = onto[nom_classe]
        # Créez une nouvelle instance de cette classe
        nouvel_animal = classe(nom_animal)


# On ajoute les images à notre ontologie.
for i in range(len(tableau_image_reconnue)):
    nom_image = tableau_image_reconnue[i][0].replace(" ", "_")  # Remplacer les espaces par des underscores
    image_instance = Image(nom_image)
    if tableau_image_reconnue[i][3] == 1:
        print("L'image", nom_image, "a été reconnue")
        image_instance.Reconnue.append(True)
    else:
        image_instance.Reconnue.append(False)

    nom_classe = tableau_image_reconnue[i][1].replace(" ", "_")
    nom_sous_classe = tableau_image_reconnue[i][2].replace(" ", "_")

    print("nom_sous_classe", nom_sous_classe)

    nom_sous_classe = onto[nom_sous_classe]

    print(nom_sous_classe)

    classe_instance = onto.search_one(type=nom_sous_classe, iri="*"+nom_classe)

    print(classe_instance)
    if classe_instance is None:
        classe_instance = nom_sous_classe(nom_classe)

    image_instance.Contient.append(classe_instance)


onto.save(file="FF3_animalia.owl", format="rdfxml")

L'image n10_01 a été reconnue
nom_sous_classe big_cat
animalia.big_cat
animalia.lion
nom_sous_classe big_cat
animalia.big_cat
animalia.lion


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation




On enregistre notre ontologie dans un fichier .owl

In [5]:
#onto.save(file="FF3_animalia.owl", format="rdfxml")